# Spacy installation and setup

In [0]:
%pip install spacy

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
!python -m spacy download en_core_web_sm 

     |█████████                       | 

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 12.8 MB 4.4 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-65e6e14c-63dd-40c1-831f-7a9c3ed5fa4e/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


# Loading plot summaries and processing it

In [0]:
plot_summaries = sc.textFile('dbfs:/FileStore/tables/plot_summaries.txt') 

In [0]:
plot_summaries.take(2)

Out[4]: ["23890098\tShlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.",
 '31186339\tThe nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual Hunger Games. The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker\'s son who once gave Katniss bread when she was starving, is the other District 12 tribute. Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy. He warns them about the "Career" tributes who train intensively at special 

In [0]:
plot_summaries_by_id = plot_summaries.map(lambda x: (x.split('\t')[0], x.split('\t')[1]))

In [0]:
plot_summaries_by_id.take(3)

Out[6]: [('23890098',
  "Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all."),
 ('31186339',
  'The nation of Panem consists of a wealthy Capitol and twelve poorer districts. As punishment for a past rebellion, each district must provide a boy and girl  between the ages of 12 and 18 selected by lottery  for the annual Hunger Games. The tributes must fight to the death in an arena; the sole survivor is rewarded with fame and wealth. In her first Reaping, 12-year-old Primrose Everdeen is chosen from District 12. Her older sister Katniss volunteers to take her place. Peeta Mellark, a baker\'s son who once gave Katniss bread when she was starving, is the other District 12 tribute. Katniss and Peeta are taken to the Capitol, accompanied by their frequently drunk mentor, past victor Haymitch Abernathy. He warns them about the "Career" tributes who train intensively 

## Removing Stop words

In [0]:
# Import library
import spacy

# Load the language model
nlp = spacy.load("en_core_web_sm")

In [0]:
import string

In [0]:
# removing stop words, punctuations and spaces
nonstop_summaries = plot_summaries_by_id.map(lambda x: (x[0], [token.text for token in nlp(x[1]) if not token.is_stop and token.text not in string.punctuation and token.text!=' ']))

In [0]:
nonstop_summaries.take(2)

Out[19]: [('23890098',
  ['Shlykov',
   'hard',
   'working',
   'taxi',
   'driver',
   'Lyosha',
   'saxophonist',
   'develop',
   'bizarre',
   'love',
   'hate',
   'relationship',
   'despite',
   'prejudices',
   'realize',
   'different']),
 ('31186339',
  ['nation',
   'Panem',
   'consists',
   'wealthy',
   'Capitol',
   'poorer',
   'districts',
   'punishment',
   'past',
   'rebellion',
   'district',
   'provide',
   'boy',
   'girl',
   'ages',
   '12',
   '18',
   'selected',
   'lottery',
   'annual',
   'Hunger',
   'Games',
   'tributes',
   'fight',
   'death',
   'arena',
   'sole',
   'survivor',
   'rewarded',
   'fame',
   'wealth',
   'Reaping',
   '12',
   'year',
   'old',
   'Primrose',
   'Everdeen',
   'chosen',
   'District',
   '12',
   'older',
   'sister',
   'Katniss',
   'volunteers',
   'place',
   'Peeta',
   'Mellark',
   'baker',
   'son',
   'gave',
   'Katniss',
   'bread',
   'starving',
   'District',
   '12',
   'tribute',
   'Katniss',
   

## Calculating TF

In [0]:
# get the format ((doc_id, term),1)
tf_keyVal = nonstop_summaries.flatMap(lambda x: [((x[0], term), 1) for term in x[1]]).reduceByKey(lambda x, y: x + y)

Output will be like `[((123546, 'Shane'),4), ((423546, 'love'),5).....]`

In [0]:
tf_keyVal.take(2)

Out[22]: [(('31186339', 'arena'), 2), (('2462689', 'stunt'), 1)]

## Calculating DF

Working on getting the below format:
$$(term_i, [ [(movie_j, tf_{i,j})], 1])$$

In [0]:
idf_keyVal = tf_keyVal.map(lambda x: (x[0][1], [[(x[0][0], x[1])], 1]))

To reduce is it in following format: $$(term_i, [[(movie_j, tf_{i,j}), (movie_{j+1}, tf_{i,j+1})....], df_i])$$

In [0]:
tf_idf_keyVal = idf_keyVal.reduceByKey(lambda x, y: [x[0] + y[0], x[1] + y[1]])

In [0]:
tf_idf_keyVal.take(5)

Out[27]: [('unstable',
  [[('1952976', 1),
    ('1580851', 1),
    ('1059701', 1),
    ('25456956', 1),
    ('11558326', 1),
    ('25625475', 1),
    ('3104623', 1),
    ('14827680', 2),
    ('1682270', 1),
    ('2750304', 1),
    ('17905510', 1),
    ('28212637', 1),
    ('24607608', 1),
    ('1565282', 1),
    ('5080406', 1),
    ('31404359', 1),
    ('22112974', 1),
    ('34596327', 1),
    ('26163439', 1),
    ('19859820', 1),
    ('28859296', 1),
    ('1343517', 1),
    ('536521', 1),
    ('4348164', 1),
    ('3418109', 1),
    ('5743048', 1),
    ('28697022', 1),
    ('15530723', 1),
    ('17687304', 1),
    ('18664245', 1),
    ('23199746', 2),
    ('213472', 1),
    ('20225956', 1),
    ('183769', 1),
    ('29836849', 1),
    ('378213', 1),
    ('3917891', 1),
    ('2102508', 1),
    ('5988879', 1),
    ('32381897', 1),
    ('26156614', 1),
    ('6503761', 1),
    ('2543730', 1),
    ('34385739', 1),
    ('23494455', 1),
    ('226842', 1),
    ('12757559', 1),
    ('2359777', 1

In [0]:
# number of documents N
N = plot_summaries_by_id.count()

In [0]:
N

Out[30]: 42306

In [0]:
import math

In [0]:
Wij = tf_idf_keyVal.flatMap(lambda x: [((x[0], tf[0]), round(tf[1]*math.log(N/x[1][1]),3)) for tf in x[1][0]])

$$
W_{ij}\hspace{1mm}is\hspace{1mm} our\hspace{1mm} TF-IDF\hspace{1mm} value\hspace{1mm} for\hspace{1mm} i^{th}\hspace{1mm} word\hspace{1mm} and\hspace{1mm} j^{th}\hspace{1mm} document.
$$

In [0]:
Wij.take(10)

Out[42]: [(('unstable', '1952976'), 5.081),
 (('unstable', '1580851'), 5.081),
 (('unstable', '1059701'), 5.081),
 (('unstable', '25456956'), 5.081),
 (('unstable', '11558326'), 5.081),
 (('unstable', '25625475'), 5.081),
 (('unstable', '3104623'), 5.081),
 (('unstable', '14827680'), 10.161),
 (('unstable', '1682270'), 5.081),
 (('unstable', '2750304'), 5.081)]

In [0]:
search_term = 'God'

In [0]:
# finding top 10 ids for a given search term
docs = Wij.filter(lambda x: x[0][0]==search_term)

In [0]:
docs.take(10)

Out[54]: [(('God', '33388421'), 11.927),
 (('God', '12121822'), 3.976),
 (('God', '5600770'), 11.927),
 (('God', '1373876'), 3.976),
 (('God', '1231877'), 3.976),
 (('God', '9653767'), 19.878),
 (('God', '6768551'), 3.976),
 (('God', '24486913'), 3.976),
 (('God', '4461403'), 3.976),
 (('God', '28557670'), 3.976)]

In [0]:
top10 = docs.sortBy(lambda x: -x[1])

In [0]:
top10List = top10.take(10)

In [0]:
top10Ids = [id[0][1] for id in top10List]
top10IdAndWeights = [(id[0][1], id[1]) for id in top10List]

# Time for looking up the top 10 movie names

In [0]:
lookupdata = sc.textFile('dbfs:/FileStore/tables/movie_metadata.tsv')

In [0]:
movie_id_name = lookupdata.map(lambda x: (x.split('\t')[0], x.split('\t')[2])).filter(lambda x: x[0] in top10Ids)

In [0]:
movie_id_name_list = movie_id_name.take(10)

In [0]:
movie_id_weight = dict(zip([x[0][1] for x in top10List], [x[1] for x in top10List]))

In [0]:
movie_id_weight

Out[64]: {'35984240': 71.561,
 '1940533': 59.634,
 '237303': 59.634,
 '3511101': 55.658,
 '1065214': 51.683,
 '2576917': 39.756,
 '1130769': 39.756,
 '98991': 39.756,
 '15525072': 39.756,
 '652437': 39.756}

In [0]:
ranked_movie_as_per_serach = sorted(movie_id_name_list,key=lambda x: -movie_id_weight[x[0]])

In [0]:
ranked_movie_as_per_serach

Out[71]: [('35984240', 'Oh My God'),
 ('1940533', 'Oh, God!'),
 ('237303', 'Bruce Almighty'),
 ('3511101', 'Skidoo'),
 ('1065214', 'Amadeus'),
 ('2576917', 'Evan Almighty'),
 ('1130769', 'The Messenger: The Story of Joan of Arc'),
 ('15525072', 'Oh, God! You Devil'),
 ('652437', 'Armour of God'),
 ('98991', 'Dogma')]